# Calculating Headways for CTA Buses - work in progress

NOTE: Headways for late night routes are not accurate yet - See the Needs Fixing section below.

### What This Does

This code calculates headways for each CTA bus stop on a given route and pattern.  It provides times when buses stopped with headways for every bus on a given route, plus summary stats for an entire route (min, max, mean, median, 25th percentile, and 75th percentile headways).

### To Do Next

- Fix late night headway issue

- Add summary stats to the stops geodataframe so they show up in the visualizations when you hover over a stop

- See what happens if you try to loop through many - or all - routes and compile a larger headways dataframe

- Investigate EWT calcs Sean MacMullan found: https://www.trapezegroup.com.au/resources/infographic-how-to-calculate-excess-waiting-time/ 

### Notes

One bus route can be made up of several patterns.  Headways are calculated for all buses running the same direction on a given route at a particular stop, regardless which pattern the bus is on.   

Vehicle data comes from the Chi Hack Night Ghost Buses breakout team: https://ghostbuses.com/about
This provides location information in 5 minute intervals for every CTA bus.  It includes
data on which route (rt) and pattern (pid) the bus was running, along with the vehicle's distance along the pattern (pdist) and a timestamp.  

Pattern data comes from the CTA's API directly. This tells us which stops are found along
a given pattern and the distance along the pattern where each stop is located.

Combining the datasets above, the general strategy is:

1. Turn vehicle data into intervals:  Time and distance are recorded at the start and end of each 5-minute interval.

2. For a given stop and pattern, find all intervals where a vehicle on that pattern reached or pased the stop.

3. Estimate the time each bus acutally reached the stop through interpolation.  The interval gives time and distance location along a given pattern before and after the bus arrived at the stop.  The CTA's pattern data tells us where the bus stop falls along the pattern.  Stop times are estimated assuming the vehicle travels a constant spaeed througout the interval.

4. Combine all stop times for buses running the same direction at a particular stop.

5. Calculate headways between buses based on stop times.

6. Calculate summary statistics on headways.


### Needs Fixing

Address late night routes. 

- Some buses run past midnight, so the first bus of the calendar day appears just after midnight. 
- Many of these have no scheduled service in the early morning hours, but the unscheduled hours show up as a long headway.  And different directions of travel may begin and end service at different times. 
    - Need to change the range of times where data is used so it doesn't assume buses should be running during their off time  
    - (Example: see first bus/last bus info on the 55 bus here: https://www.transitchicago.com/bus/55/)
        - Parts of this route run 24 hours
        - Parts run 5am to 2am
        - Parts run 4:10 am to 1:10am
        - First and last buses reach mid-route stops later than these ranges. Buses continue to the end of their route
        - not in service times between 5am and 2am show up in the code below as long headways
 
- Possible Approach?

    - Combine 2 days of data: the "main" date being looked at and the following date for any schedules that splill into overnight.  How to choose a new range that starts and ends with the out-of-service period?

    - Investigate the schedule data from CTA. Can we extract actual start/end times for a route/direction, adding a buffer of extra time on the end to capture buses finishing their routes at end of the schedule? 

- Also, this code currently eliminates all intervals where one timestamp is before midnight and the next is after, since it uses one calendar day of data.  Missing one inverval per day is less critical than the issue above, and it would likely be resolved by the same fixes.




In [1]:
import requests
from dotenv import load_dotenv
import pandas as pd
import geopandas as gpd
from shapely import Point, LineString
import datetime as dt
import numpy as np

In [2]:
# Get API key from the .env file
load_dotenv()
API_KEY = os.getenv('API_KEY')

### Functions

In [3]:
def get_chn_vehicles(datestring:str) -> pd.DataFrame:
    """Datestring must be in YYYY-MM-DD format. Returns vehicle data scraped by the chn
    ghost bus team for all CTA buses on the specified date."""

    chn_data_source = f'https://chn-ghost-buses-public.s3.us-east-2.amazonaws.com/bus_full_day_data_v2/{datestring}.csv'

    vehicles = pd.read_csv(
        chn_data_source, dtype={
            'vid':'int',
            'tmstmp':'str',
            'lat':'float',
            'lon':'float',
            'hdg':'int',
            'pid':'int',
            'rt':'str',
            'pdist':'int',
            'des':'str',
            'dly':'bool',
            'tatripid':'str',
            'origatripno':'int',
            'tablockid':'str',
            'zone':'str',
            'scrape_file':'str',
            'data_hour':'int',
            'data_date':'str'
            }
        )

    return vehicles


In [4]:
def get_patterns(vehicles:pd.DataFrame, rt:str) -> pd.DataFrame:
    '''Get patterns data from the CTA's bus tracker API for all
    vehiles in the vehicles data. Return patterns as a dataframe'''

    df_output = pd.DataFrame()

    # filter vehicles to the specified route
    rt_vehicles = vehicles.loc[vehicles['rt'] == rt]

    # list pid values included in the route
    pid_list = list(rt_vehicles['pid'].unique())

    # convert pids to strings
    pid_list = [str(i) for i in pid_list]

    # split pid_list into chunks of 10 (limit of the API):
    start = 0
    end = len(pid_list)
    step = 10
    for i in range(start, end, step):
        pid_list_chunk = pid_list[i:i+step]
        pid_string = ','.join(pid_list_chunk)

        # get data from CTA's feed
        api_url = f'http://www.ctabustracker.com/bustime/api/v2/getpatterns?key={API_KEY}&pid={pid_string}&format=json'
        response = requests.get(api_url)
        patterns = response.json()

        # convert json to dataframe
        df_patterns = pd.DataFrame(patterns['bustime-response']['ptr'])

        # add to the output dataframe
        df_output = pd.concat([df_output, df_patterns])


    # convert pt column values to dataframes for each pattern containing that pattern's points
    df_output['pt'] = df_output['pt'].apply(lambda x: pd.DataFrame(x))
    
    return df_output


In [5]:

def get_pattern_linestrings(patterns:pd.DataFrame) -> gpd.GeoDataFrame:
    '''Use the get_patterns function to generate patterns input for this function.
    Return patterns as a geodataframe with linestring geometry for each pattern'''

    df_patterns = patterns.copy()

    # Turn points into linestrings
    geometry_linestrings = []
    for p in df_patterns['pt']:
        p.sort_values('seq', inplace=True)
        linestring_points = list(zip(p['lon'],p['lat']))

        # generate linestring using all points
        linestring = LineString(linestring_points)
        geometry_linestrings.append(linestring)

    # Create a geodataframe for the patterns using the linestring geometry
    gdf_patterns = gpd.GeoDataFrame(df_patterns, geometry=geometry_linestrings).set_crs(epsg=4326)

    # Drop the original pt column
    gdf_patterns.drop(['pt'], axis=1, inplace=True)

    return gdf_patterns


In [6]:
def get_pattern_stops(patterns) -> gpd.GeoDataFrame:
        '''Get the patterns data from the CTA's bus tracker API 
        for a specified route. Return patterns as a geodataframe 
        with point geomtry, one point per bus stop on each pattern
        associated with the route. Note that stops serving multiple
        patterns will be listed multiple times, once for each pattern
        with the seq and pdist values specific to that pattern.'''

        # get patterns for the route
        df_patterns = patterns.copy()

        # set up a geodataframe to contain stops
        gdf_route_stops = gpd.GeoDataFrame()

        # consider the pid column (pattern ID) and the pt column (dataframe contaning
        # points along the pattern)
        for pid, pt in zip(df_patterns['pid'],df_patterns['pt']):
                # sort points sequentially
                pt.sort_values('seq', inplace=True)
                # add the pattern id to each point's data
                pt['pid']=pid
                # add the pattern direction to each point's data
                rtdir = df_patterns['rtdir'].loc[df_patterns['pid'] == pid].tolist()[0]
                pt['rtdir'] = rtdir
                # filter to only show stop points
                stops = pt[pt['typ']=='S']
                # zip lat/lon data to get coordinate pairs
                coords = list(zip(stops['lon'],stops['lat']))
                # turn coordinates into point geometry
                geometry = [Point(c) for c in coords]
                # generate a geodataframe for the stops in this pattern
                gdf_pattern_stops = gpd.GeoDataFrame(stops,geometry=geometry).set_crs(epsg=4326)
                # add this pattern's stops to the dataframe containing all stops on the route
                gdf_route_stops = pd.concat([gdf_route_stops, gdf_pattern_stops])

        return gdf_route_stops


In [7]:
def get_vehicle_intervals(vehicles:pd.DataFrame, rt:str) -> pd.DataFrame:

    """vehicles should be a CHN scraped dataframe including vehicle ids (vid),
    timestamps (tmstmp), pattern ids (pid), and pattern distances (pdist) for each vehicle at 
    various times.  See get_chn_vehicles function. Output is a dataframe with each row representing
    an interval between two points in time and space for one vehicle. Columns are added
    for each interval's start time, end time, start pdist, and end pdist."""

    df_vehicles = vehicles.copy()

    # filter to the specified route
    df_vehicles = df_vehicles.loc[df_vehicles['rt'] == rt]

    # Set up dataframe to contain final fomratted data
    df_output = pd.DataFrame()

    # End time for each interval as a timestamp
    df_vehicles['end_time'] = pd.to_datetime(df_vehicles['tmstmp'],infer_datetime_format=True)
    vid_list = df_vehicles['vid'].unique().tolist()

    # End location for each interval
    df_vehicles['end_pdist'] = df_vehicles['pdist']

    for vid in vid_list:

        # pare data down to a single vehicle
        df_vehicle = df_vehicles.loc[df_vehicles['vid'] == vid]

        # handle each pattern separately
        pid_list = df_vehicle['pid'].unique().tolist()
        for p in pid_list:
            df_vehicle_pattern = df_vehicle.loc[df_vehicle['pid']==p].copy()
            # sort by time (it should be sorted already, but just in case)
            df_vehicle_pattern.sort_values(by=['end_time'], inplace=True)

            # Create a start time based on the previous tinmestamp
            end_times = df_vehicle_pattern['end_time'].tolist()
            start_times = np.roll(end_times,shift=1)
            df_vehicle_pattern['start_time'] = start_times

            # Create a start pattern distance based on the previous pdist
            end_distances = df_vehicle_pattern['end_pdist'].tolist()
            start_distances = np.roll(end_distances,shift=1)
            df_vehicle_pattern['start_pdist'] = start_distances

            # Remove the first interval since we don't have real start
            # time or location data for it
            df_vehicle_pattern = df_vehicle_pattern.iloc[1:]

            # add data to the full output dataframe
            df_output = pd.concat([df_output, df_vehicle_pattern])

    return df_output



In [8]:
        # Interpolate estimated times the bus arrived at a stop
        def interpolate_stop_time(
            stop_pdist:int, 
            start_time:pd.Timestamp, 
            end_time:pd.Timestamp, 
            start_pdist:int, 
            end_pdist:int
            ) -> pd.Timestamp:

            # How far into the interval distance is the bus stop?
            # stop distance from beginning of interval / full interval distance
            dist_ratio = (stop_pdist-start_pdist)/(end_pdist-start_pdist)

            # estimated bus stop time, assuming it traveled at a steady
            # speed throughout the interval
            est_stop_time = start_time + (end_time - start_time)*dist_ratio

            # round estimated stop time to the nearest minute
            est_stop_time = est_stop_time.round(freq='T')

            return est_stop_time

In [9]:
def get_stoptimes(rt:str, vehicles:pd.DataFrame) -> pd.DataFrame:
    '''Input:  vehicles can be obtained using the get_chn_vehicles function. Output is a
    pandas dataframe that includes estimated bus stop times for all buses on the
    specified route.'''

    df_output = pd.DataFrame()

    vehicle_intervals = get_vehicle_intervals(vehicles, rt)

    df_patterns = get_patterns(vehicles, rt)

    # get all stops on this route, including all patterns
    gdf_stops = get_pattern_stops(df_patterns)

    # Consider each combination of stop and pattern
    for stpid, pid, rtdir in list(zip(gdf_stops['stpid'],gdf_stops['pid'], gdf_stops['rtdir'])):

        # get a single stop, including all patterns for this route that use the stop
        gdf_this_stop_pattern = gdf_stops.loc[(gdf_stops['stpid'] == stpid) & (gdf_stops['pid'] == pid)]
        if len(gdf_this_stop_pattern) == 0:
            continue
            
        # Find the bus stop's distance along the pattern
        pdist_this_stop = gdf_this_stop_pattern['pdist'].tolist()[0]

        # find the intervals that are on this pattern
        df_this_pattern_intervals = vehicle_intervals.loc[vehicle_intervals['pid'] == pid]
        if len(df_this_pattern_intervals) == 0:
            continue

        # Filter for intervals that start ahead of the stop location and end at or beyond the stop
        def filter_intervals(stop_dist:int, start_pdist:int, end_pdist:int):
            return (start_pdist < stop_dist) & (end_pdist >= stop_dist)
    
        # Create filter for the intervals we're working on
        interval_filter = df_this_pattern_intervals.apply(
            lambda x: filter_intervals(pdist_this_stop, x['start_pdist'], x['end_pdist']), axis=1
            )
        
        # apply the filter
        df_this_pattern_stop_intervals = df_this_pattern_intervals.loc[interval_filter]
        if len(df_this_pattern_stop_intervals) == 0:
            continue

        # Add stpid, pdist, and rtdir to the data
        df_this_pattern_stop_intervals['stpid'] = stpid
        df_this_pattern_stop_intervals['stop_pdist'] = int(pdist_this_stop)
        df_this_pattern_stop_intervals['rtdir'] = rtdir


        # Estimate time each bus passed the stop (interpolated based on data at start and
        # end of the interval)
        df_this_pattern_stop_intervals['est_stop_time'] = df_this_pattern_stop_intervals.apply(
            lambda x: interpolate_stop_time(
                pdist_this_stop, 
                x['start_time'], 
                x['end_time'], 
                x['start_pdist'], 
                x['end_pdist']), axis=1
            )

        # Add the intervals with stop times to the full output dataframe
        df_output = pd.concat([df_output, df_this_pattern_stop_intervals])

    return df_output



In [10]:

def get_headways(rt:str, vehicles:pd.DataFrame) -> pd.DataFrame:

        '''vehicles data can be obtained using the get_chn_vehicles function.
        Output is a dataframe containing stop times and calculated headways 
        for every bus stop on the route.'''

        df_output = pd.DataFrame()

        # Times buses stopped at each stop on the route
        df_stoptimes = get_stoptimes(rt, vehicles).copy()

        # consider all buses stopping at a given stop moving int he same direction
        for stpid, rtdir in set(zip(df_stoptimes['stpid'], df_stoptimes['rtdir'])):

                # filter data
                df_stop_direction = df_stoptimes.loc[(df_stoptimes['stpid'] == stpid) & (df_stoptimes['rtdir'] == rtdir)]

                # Sort chronologically
                df_stop_direction.sort_values(by='est_stop_time',ascending=True, inplace=True)

                # list stop times in chronological order
                stop_times = df_stop_direction['est_stop_time'].tolist()

                # calculate previous stop time for each line
                prev_stop_times = np.roll(stop_times,1)
                df_stop_direction['previous_stop_time'] = prev_stop_times

                # calculate headway
                df_stop_direction['est_headway'] = (
                df_stop_direction['est_stop_time'] - df_stop_direction['previous_stop_time']
                )
                
                # drop previous stop time column, no longer needed
                df_stop_direction = df_stop_direction.drop('previous_stop_time', axis=1)

                # Remove headway from the first bus in the dataset since we don't have the 
                # previous bus to compare with
                df_stop_direction['est_headway'].iloc[0] = None
                
                df_output = pd.concat([df_output, df_stop_direction])

        return df_output



In [11]:
def get_headway_stats(headways:pd.DataFrame) -> dict:
    est_headways = headways['est_headway']
    stats = {
        'mean':est_headways.mean(),
        'max':est_headways.max(),
        'min':est_headways.min(),
        '25th_pctile':est_headways.quantile(0.25), # 25th percentile
        'median':est_headways.median(), # 50th percentile
        '75th_pctile':est_headways.quantile(0.75)
    }
    return stats




## Try it out

In [15]:
vehicles = get_chn_vehicles('2023-01-11')

In [25]:
%%capture --no-display 
# turn off warnings

# Check out the Garfield bus (55)
headways = get_headways('55',vehicles)

headways

,vid,tmstmp,lat,lon,hdg,pid,rt,des,pdist,dly,...,data_date,end_time,end_pdist,start_time,start_pdist,stpid,stop_pdist,rtdir,est_stop_time,est_headway
319,8033,20230111 00:07,41.793881,-87.649419,87,5424,55,Museum of Science & Industry,29678,False,...,2023-01-11,2023-01-11 00:07:00,29678,2023-01-11 00:02:00,23686,10526,24335,Eastbound,2023-01-11 00:03:00,NaT
1110,8004,20230111 00:27,41.793861,-87.652756,89,5424,55,Museum of Science & Industry,28769,False,...,2023-01-11,2023-01-11 00:27:00,28769,2023-01-11 00:22:00,21862,10526,24335,Eastbound,2023-01-11 00:24:00,0 days 00:21:00
1950,8221,20230111 00:52,41.793613,-87.668870,88,5424,55,Museum of Science & Industry,24370,False,...,2023-01-11,2023-01-11 00:52:00,24370,2023-01-11 00:47:00,19593,10526,24335,Eastbound,2023-01-11 00:52:00,0 days 00:28:00
2739,1568,20230111 01:22,41.793813,-87.654249,89,5424,55,Museum of Science & Industry,28362,False,...,2023-01-11,2023-01-11 01:22:00,28362,2023-01-11 01:17:00,22067,10526,24335,Eastbound,2023-01-11 01:19:00,0 days 00:27:00
3294,8033,20230111 01:52,41.793921,-87.647007,89,5424,55,Museum of Science & Industry,30331,False,...,2023-01-11,2023-01-11 01:52:00,30331,2023-01-11 01:47:00,23684,10526,24335,Eastbound,2023-01-11 01:47:00,0 days 00:28:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163732,1481,20230111 22:17,41.793154,-87.727476,90,5424,55,Museum of Science & Industry,8164,False,...,2023-01-11,2023-01-11 22:17:00,8164,2023-01-11 22:12:00,891,10496,5367,Eastbound,2023-01-11 22:15:00,0 days 00:19:00
165249,8021,20230111 22:37,41.793087,-87.735657,89,5424,55,Museum of Science & Industry,5902,False,...,2023-01-11,2023-01-11 22:37:00,5902,2023-01-11 22:32:00,1,10496,5367,Eastbound,2023-01-11 22:37:00,0 days 00:22:00
166961,8041,20230111 23:02,41.793186,-87.725108,89,5424,55,Museum of Science & Industry,8816,False,...,2023-01-11,2023-01-11 23:02:00,8816,2023-01-11 22:57:00,2953,10496,5367,Eastbound,2023-01-11 22:59:00,0 days 00:22:00
168172,1570,20230111 23:22,41.793126,-87.730512,88,5424,55,Museum of Science & Industry,7336,False,...,2023-01-11,2023-01-11 23:22:00,7336,2023-01-11 23:17:00,441,10496,5367,Eastbound,2023-01-11 23:21:00,0 days 00:22:00


In [26]:
# Headway stats for the 55 bus (note the max here is incorrect - it's based on 
# early morning hours when the bus isn't running)

stats = get_headway_stats(headways)
stats

{'mean': Timedelta('0 days 00:14:14.313860252'),
 'max': Timedelta('0 days 02:59:00'),
 'min': Timedelta('0 days 00:00:00'),
 '25th_pctile': Timedelta('0 days 00:09:00'),
 'median': Timedelta('0 days 00:13:00'),
 '75th_pctile': Timedelta('0 days 00:18:00')}

In [31]:
# Visualize routes and stops for the 55 bus (no headway info included in visualization yet)
patterns = get_patterns(vehicles, '55')
route = get_pattern_linestrings(patterns)
stops = get_pattern_stops(patterns)

m = route.explore(color='blue', tiles='CartoDB_positron')
stops.explore(m=m, color='red')
